# Implementações

Fernanda Gomes e Igor Michels

In [1]:
using LinearAlgebra;

## Forma Padrão

In [2]:
function SEF(obj, c, A, sig, b, x_cons)
    """
    obj    : string (Min or Max)
    c      : vector of costs
    A      : matrix of coeff
    sig    : constraints' signal (vector of strings)
    b      : constraints (vector)
    x_cons : vector of constraints for x vector ("<=", ">=" or "" in relation to 0)
    """
    
    if obj == "Min"
        obj = "Max";
        c = -c;
    end
    
    for i in 1:length(x_cons)
        if x_cons[i] == "<="
            A[:, i] = -A[:, i];
            c[i] = -c[i];
        elseif x_cons[i] == ""
            A = hcat(A, -A[:, i]);
            append!(c, -c[i])
            x_cons[i] = ">=";
            append!(x_cons, [">="]);
        end
    end
    
    for i in 1:length(sig)
        if sig[i] == ">="
            n_lin, n_col = size(A);
            A = hcat(A, zeros(n_lin, 1));
            A[i, n_col + 1] = -1;
            sig[i] = "=";
            append!(x_cons, [">="]);
            append!(c, [0]);
        elseif sig[i] == "<="
            n_lin, n_col = size(A);
            A = hcat(A, zeros(n_lin, 1));
            A[i, n_col + 1] = 1;
            sig[i] = "=";
            append!(x_cons, [">="]);
            append!(c, [0]);
        end
    end
    return obj, c, A, sig, b, x_cons
end;

### Teste

In [3]:
obj = "Min";
c = [-1, 2, -4];
A = [ 1  5  3
      2 -1  2
      1  2 -1];
sig = [">=", "<=", "="];
b = [5, 4, 2];
x_cons = [">=", ">=", ""];

obj, c, A, sig, b, x_cons = SEF(obj, c, A, sig, b, x_cons)

("Max", [1, -2, 4, -4, 0, 0], [1.0 5.0 … -1.0 0.0; 2.0 -1.0 … 0.0 1.0; 1.0 2.0 … 0.0 0.0], ["=", "=", "="], [5, 4, 2], [">=", ">=", ">=", ">=", ">=", ">="])

## Iteração Simplex

In [4]:
function iteracao_simplex(c, A, b, B, N)
    x = zeros(1, size(A, 2));
    x[B] = b;
    i = findfirst(c .> 0);
    if isnothing(i)
        return B, N, x
    end
    
    coeff = A[:, i];
    t = b ./ coeff;
    if t[t .> 0] == []
        return "Ilimitado";
    end
    
    tₒ = minimum(t[t .> 0]);
    j = findfirst(t .== tₒ)
    x[i] = tₒ;
    for k in 1:length(B)
        x[B[k]] -= coeff[k] * tₒ;
    end
    
    B[j] = i;
    N = Array(1:size(A, 2));
    N = N[N .∉ Ref(B)];
    sort!(B);
    sort!(N);
    
    return B, N, x
end;

## Forma Canônica

In [5]:
function CF(c, A, b, B = nothing, N = nothing, z = 0)
    if isnothing(B)
        n_lin, n_col = size(A);
        corte = n_col - n_lin + 1;
        B = Array(corte:n_col);
        N = Array(1:(corte - 1));
    end
    
    A_B = A[:, B];
    A_B⁻¹ = inv(A_B);
    y = transpose(A_B⁻¹) * c[B];
    z = z + transpose(y) * b;
    c = c - transpose(A) * y;
    A = A_B⁻¹ * A;
    b = A_B⁻¹ * b;
    return c, A, b, B, N, z
end;

### Teste

In [6]:
obj = "Max";
c = [3, 4, 6];
A = [ 3  5 -6
      1  3 -4
     -1  1 -1];
sig = ["=", "=", "="];
b = [4, 2, -1];
x_cons = [">=", ">=", ">="];

obj, c, A, sig, b, x_cons = SEF(obj, c, A, sig, b, x_cons);
CF(c, A, b)

([0.0, 0.0, 0.0], [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0], [1.0, -1.0, -1.0], [1, 2, 3], Int64[], -7.0)

## Teste de Infactibilidade

In [7]:
function infeasibility(A, b)
    n, m = size(A);
    A_aux = hcat(A, Matrix(I, n, n));
    c = - vcat(zeros(m), ones(n));
    sig = ["=" for i in 1:n];
    x_cons = [">=" for i in 1:(m + n)];
    x, z = simplex("Max", c, A_aux, sig, b, x_cons, false, false);
    if z == 0
        return true;
    else
        return false;
    end
end;

## Implementação do Simplex

In [8]:
function simplex(obj, c, A, sig, b, x_cons, verify = true, printing = true)
    obj, c, A, sig, b, x_cons = SEF(obj, c, A, sig, b, x_cons);
    B = nothing;
    N = nothing;
    z = 0;
    
    feasible = true;
    if verify
        feasible = infeasibility(A, b);
    end
    
    if ~feasible
        println("Problema infactível");
        return "Infactível"
    end
    
    while true
        c, A, b, B, N, z = CF(c, A, b, B, N, z);
        sol = iteracao_simplex(c, A, b, B, N)
        if sol == "Ilimitado"
            println("Problema ilimitado");
            return "Ilimitado";
        else
            B, N, x = sol;
        end
        
        if c[c .> 0] == []
            if printing
                println("Ótimo em $x, com valor z = $z");
            end
            return x, z;
        end
    end
end;

### Teste

In [9]:
obj = "Max";
c = [ 2,  3,  0,  0,  0];
A = [ 1  1  1  0  0
      2  1  0  1  0
     -1  1  0  0  1];
sig = ["=", "=", "="];
b = [6, 10, 4];
x_cons = [">=", ">=", ">=", ">=", ">="];
B = nothing;
N = nothing;
z = 0;
solution = simplex(obj, c, A, sig, b, x_cons);

Ótimo em [1.0 5.0 0.0 3.0 0.0], com valor z = 17.0
